In [1]:
print( "┏━━━━━━━━━┉┅╍╌" )

import ipywidgets
import easygopigo3
import time
import threading
import picamera
import numpy ## because picamera can't just capture an image to an object
from PIL import Image as PIL_Image ## because PIL couldn't be bothered to make PIL.Image work
from di_sensors import inertial_measurement_unit ## because di_sensors couldn't be bothered to make di_sensors.inertial_measurement_unit work
import math

FRAME_INTERVAL = 0.1 ## seconds
DISTANCE_SERVO_OFFSET = 7 ## difference in degrees between servo orientation and body orientation
CAMERA_SERVO_OFFSET = -18 ## difference in degrees between servo orientation and body orientation
STOPPING_DISTANCE = 500
rename_rotational_factor = 0.30
rename_accepted_minimum_by_drivers = 6
rename_acceptable_error_percent = 8

# output = ipywidgets.Output( layout=ipywidgets.Layout( height="20em" ) )
output = ipywidgets.Output()
robot = easygopigo3.EasyGoPiGo3()
distance = robot.init_distance_sensor()
distance_servo = robot.init_servo("SERVO1")
camera = picamera.PiCamera()
camera_servo = robot.init_servo("SERVO2")
imu = inertial_measurement_unit.InertialMeasurementUnit(bus = "GPG3_AD1")

motionMonitor = None

def startMotionMonitor():
    global motionMonitor
    if motionMonitor:
        motionMonitor.cancel()
        with output:
            print( "motion monitoring continuing" )
    else:
        with output:
            print( "motion monitoring starting" )
    d = getDistance()
    h = getHeading()
    if d <= STOPPING_DISTANCE:
        with output:
            print( "Stopping Distance Reached!")
            doStop()
            getPhoto()
    motionMonitor = threading.Timer( FRAME_INTERVAL, startMotionMonitor )
    motionMonitor.start()

def stopMotionMonitor():
    global motionMonitor
    if motionMonitor:
        motionMonitor.cancel()
    getDistance()
    if motionMonitor:
        with output:
            print( "motion monitoring stopping" )
    else:
        with output:
            print( "motion monitoring staying stopped" )
    motionMonitor = None

def aimBody( degrees_east_of_north ):
    current_heading = getHeading()  ## apparently this can't be called inside of with output
    with output:
        if degrees_east_of_north == 180.0:
            heading_diff = (degrees_east_of_north - abs(current_heading)) * (-1 if current_heading < 0 else 1)
            error = abs(heading_diff / degrees_east_of_north) * 100
        else:
            heading_diff = degrees_east_of_north - current_heading
            error = abs(heading_diff / 180) * 100
        print( "Change heading by "+str(heading_diff)+" ±"+str(error)+"% degrees" )
        how_much_to_rotate = int(heading_diff * rename_rotational_factor)
        if error >= rename_acceptable_error_percent and abs(how_much_to_rotate) >= rename_accepted_minimum_by_drivers:
            print( "Rotate by "+str(how_much_to_rotate) )
            robot.turn_degrees(how_much_to_rotate, blocking = True)
        else:
            print( "Calls for rotation of "+str(how_much_to_rotate)+", less than the minimum of "+str(rename_accepted_minimum_by_drivers) )

def doTurnNorth(b=None):
    with output:
        aimBody( 0 )
def doTurnWest(b=None):
    with output:
        aimBody( -90 )
def doTurnEast(b=None):
    with output:
        aimBody( 90 )
def doTurnSouth(b=None):
    with output:
        aimBody( 180 )

def doTurnLeft(b=None):
    with output:
        print( "Turning Left" )
        robot.left()
        startMotionMonitor()
def doTurnRight(b=None):
    with output:
        print( "Turning Right" )
        robot.right()
        startMotionMonitor()

def doGoForward(b=None):
    with output:
        print( "Going Forward" )
        robot.forward()
        startMotionMonitor()
def doGoBackward(b=None):
    with output:
        print( "Going Backward" )
        robot.backward()
        startMotionMonitor()

def doStop(b=None):
    with output:
        print( "Stopping" )
        robot.stop()
        stopMotionMonitor()

def aimDistance( degrees_right = 0 ):
    control_angle = 90 + DISTANCE_SERVO_OFFSET - degrees_right
    print( "aimDistance( "+str(degrees_right)+" ): control_angle = "+str(control_angle) )
    distance_servo.rotate_servo( control_angle )  ## TODO: impose limits (don't strain motor); 0-?

def aimCamera( degrees_right = 0 ):
    control_angle = 90 + CAMERA_SERVO_OFFSET - degrees_right
    print( "aimCamera( "+str(degrees_right)+" ): control_angle = "+str(control_angle) )
    camera_servo.rotate_servo( control_angle )  ## TODO: impose limits (don't strain motor); 6-?

def doPrepServos(b=None):
    with output:
        print( repr( b ) )
        print( "Prepping Servos" )
        aimDistance( 0 ) ## set distance servo to middle position (forward)
        aimCamera( 0 ) ## set camera servo to middle position (forward)
def doParkServos(b=None):
    with output:
        print( repr( b ) )
        print( "Parking Servos" )
        aimDistance( 90 + DISTANCE_SERVO_OFFSET ) ## set distance servo to far right position
        aimCamera( 84 + CAMERA_SERVO_OFFSET ) ## set distance servo to far right position

def getDistance(b=None):
    d = None
    with output:
        d = distance.read_mm()
        print( "Distance is "+str(d) )
    return d

def getPhoto(b=None):
    image = None
    with output:
        pixels = numpy.empty((480, 640, 3), dtype = numpy.uint8)
        camera.resolution = (640, 480)
        camera.capture( pixels, format = 'rgb' )
        image = PIL_Image.fromarray(pixels)
        display( image )  ##  analogous to print, but accepts images
    return image

def getHeading(b=None):  ## returns degrees east of north
    heading = None
    with output:
        x, y, z = imu.read_magnetometer()
        print( "Magnetometer: x="+str(x)+" y="+str(y)+" z="+str(z) )
        heading = -math.atan2(x, -z) * 180 / math.pi
        if heading < 0:
            heading += 360
        elif heading > 360:
            heading -= 360
        if 180 < heading <= 360:
            heading -= 360
        print( "Heading is "+str(heading) )
    return heading

def doExit(b=None):
    with output: print( "EXITING" )
    doParkServos()
    global controls
    controls.close()
    robot.reset_all()
    print( "EXITING" ) ## Why doesn't this work?
    exit()

def controls():
    def make_button( label, on_click, layout=ipywidgets.Layout( width="10em" ), color='#888888' ):
        button = ipywidgets.Button( description=label, layout=layout )
        button.style.button_color = color
        button.on_click(on_click)
        return button

    bNorth = make_button( label="Turn North", on_click=doTurnNorth )
    bWest  = make_button( label="Turn West",  on_click=doTurnWest  )
    bEast  = make_button( label="Turn East",  on_click=doTurnEast  )
    bSouth = make_button( label="Turn South", on_click=doTurnSouth )

    nsew_box = ipywidgets.widgets.VBox( [
        ipywidgets.widgets.HBox( [ bNorth ],      layout=ipywidgets.Layout( justify_content="center" ) ),
        ipywidgets.widgets.HBox( [ bWest, bEast ] ),
        ipywidgets.widgets.HBox( [ bSouth ],      layout=ipywidgets.Layout( justify_content="center" ) ),
    ], layout=ipywidgets.Layout( width="fit-content" ) )

    bForward  = make_button( label="Go Forward",  on_click=doGoForward  )
    bLeft     = make_button( label="Turn Left",   on_click=doTurnLeft   )
    bStop     = make_button( label="STOP",        on_click=doStop,      color='red' )
    bRight    = make_button( label="Turn Right",  on_click=doTurnRight  )
    bBackward = make_button( label="Go Backward", on_click=doGoBackward )

    move_box = ipywidgets.widgets.VBox( [
        ipywidgets.widgets.HBox( [ bForward ],            layout=ipywidgets.Layout( justify_content="center" ) ),
        ipywidgets.widgets.HBox( [ bLeft, bStop, bRight ] ),
        ipywidgets.widgets.HBox( [ bBackward ],           layout=ipywidgets.Layout( justify_content="center" ) ),
    ], layout=ipywidgets.Layout( width="fit-content" ) )
    
    bDistance = make_button( label="Distance", on_click=getDistance )
    bPhoto    = make_button( label="Photo",    on_click=getPhoto    )
    bHeading  = make_button( label="Heading",  on_click=getHeading  )
    
    data_box = ipywidgets.widgets.VBox( [ bDistance, bPhoto, bHeading ], layout=ipywidgets.Layout( justify_content="center" ) )
    
    bPrep = make_button( label="Prep", on_click=doPrepServos )
    bPark = make_button( label="Park", on_click=doParkServos )
    bExit = make_button( label="Exit", on_click=doExit       )
    
    misc_box = ipywidgets.widgets.VBox( [ bPrep, bPark, bExit ], layout=ipywidgets.Layout( justify_content="center" ) )

    spacer = ipywidgets.widgets.VBox( layout=ipywidgets.Layout( width="4em" ) )

    return ipywidgets.widgets.VBox( [
        ipywidgets.widgets.HBox( [ nsew_box, spacer, move_box, spacer, data_box, spacer, misc_box ] ),
        ipywidgets.widgets.HBox( layout=ipywidgets.Layout( height="2em" ) ),
        ipywidgets.widgets.HBox( [ output ] ),
    ] )

doPrepServos()
getDistance()
controls = controls()
display( controls )
with output:
    print( "Motion Monitoring Updates every "+str(FRAME_INTERVAL)+" seconds while moving" )
getDistance()
getHeading()
# doGoForward()
print( " ╌╍┅┉━━━━━━━━━┛" )


┏━━━━━━━━━┉┅╍╌


VBox(children=(HBox(children=(VBox(children=(HBox(children=(Button(description='Turn North', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')),), layout=Layout(justify_content='center')), HBox(children=(Button(description='Turn West', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')), Button(description='Turn East', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')))), HBox(children=(Button(description='Turn South', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')),), layout=Layout(justify_content='center'))), layout=Layout(width='fit-content')), VBox(layout=Layout(width='4em')), VBox(children=(HBox(children=(Button(description='Go Forward', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')),), layout=Layout(justify_content='center')), HBox(children=(Button(description='Turn Left', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')), Button(description='STOP', layout=Layout(width='10em'), style=ButtonStyle(button_color='red')), Button(description='Turn Right', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')))), HBox(children=(Button(description='Go Backward', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')),), layout=Layout(justify_content='center'))), layout=Layout(width='fit-content')), VBox(layout=Layout(width='4em')), VBox(children=(Button(description='Distance', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')), Button(description='Photo', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')), Button(description='Heading', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888'))), layout=Layout(justify_content='center')), VBox(layout=Layout(width='4em')), VBox(children=(Button(description='Prep', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')), Button(description='Park', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888')), Button(description='Exit', layout=Layout(width='10em'), style=ButtonStyle(button_color='#888888'))), layout=Layout(justify_content='center')))), HBox(layout=Layout(height='2em')), HBox(children=(Output(),))))

 ╌╍┅┉━━━━━━━━━┛
